# Imagery Insights eval

> Add blockquote

This notebook demonstrates how to classify images from GCS URIs using the Gemini 1.5 Flash model via Google Cloud's Vertex AI.

## Install Required Libraries

In [ ]:
!pip install --upgrade google-cloud-bigquery google-cloud-aiplatform pandas

## Imports and Vertex AI Initialization

In [ ]:
import vertexai
from google.cloud import bigquery
from google.cloud import storage
from google import genai
from google.genai.types import Content, Part
import pandas as pd
from IPython.display import display
from collections import Counter, defaultdict
import os, re, base64
import json

In [ ]:
# --- Configuration ---
PROJECT_ID = ''  # @param {type:"string"}
REGION = 'us-central1'      # @param {type:"string"}
DATASET_ID = '' # @param {type:"string"}
TABLE_ID = '' # @param {type:"string"}
QUERY_LIMIT = 10 # @param {type:"integer"}
MODEL_FLASH = 'gemini-2.5-flash' # @param {type:"string"}
MODEL_PRO = 'gemini-2.5-pro' # @param {type:"string"}

## Initialize Vertex AI SDK

In [ ]:
# --- Initialize Vertex AI SDK ---
vertexai.init(project=PROJECT_ID, location=REGION)

print("✅ Setup Complete: Libraries installed, configured, and Vertex AI initialized.")

## Create vertex client

In [ ]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

##Compares two model outputs to see if they are semantically the same.

    Args:
        output1: The first model's JSON output.
        output2: The second model's JSON output.
        model_name: The name of the Gemini model to use for comparison.

    Returns:
        'Yes' if the outputs are semantically the same, 'No' otherwise.

In [ ]:
def judge_outputs(output1: str, output2: str) -> str:

    comparison_prompt = f"""You will be provided with two JSON objects. Your task is to determine if they are semantically the same.

    Respond with 'SAME' if they are semantically the same, and 'DIFFERENT' if they are not.

    Add key points that you saw as different.

    JSON 1:
    {output1}

    JSON 2:
    {output2}
    """
    contents = [
    comparison_prompt]

    try:

        response = client.models.generate_content(model=MODEL_FLASH, contents=contents)

        return response.text
    except Exception as e:
        print(f"Error during comparison: {e}")
        return "Comparison failed."

## Classify image with Gemini

In [ ]:
def classify_image_with_gemini(gcs_uri: str, prompt: str, model_name: str) -> str:
    """Classifies an image using a Gemini model."""

    # The 'mime_type' must be passed as a keyword argument.
    contents = [
    prompt,
    Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]

    response = client.models.generate_content(model=model_name, contents=contents)
    return response.text

## Helper function for json sanitization

In [ ]:
def _extract_json(text: str) -> str:
    """Extracts the JSON string from a text block, even if it's wrapped in markdown."""
    # Use a regex to find the content between ```json and ```
    match = re.search(r"```json\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        return match.group(1)
    # Fallback for just ```
    match = re.search(r"```\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        return match.group(1)
    # If no markdown, assume the whole string is the json
    return text

## Prompt to extract information from given images

In [ ]:
prompt = """You will be provided with a photo of a utility pole.
User Input:
{
photo_of_utility_pole
}

Follow these instructions to analyze the image:

1.  **Analyze the Image:** Carefully examine the provided image. If the image does not clearly show a utility pole, you must return the following JSON object and stop processing:
    `{\"error\": \"No utility pole detected in the image.\"}`

2.  **Identify Key Information:** If a utility pole is present, identify the following information from the image:
    *   **Material:** Determine the material the pole is made from (e.g., \"wood\", \"metal\", \"concrete\").
    *   **Type:** Determine the primary type of the pole. Choose from: \"Street light\", \"High tension power transmission\", \"electricity pole\", or \"other\".


3.  **Format the Output:** Your response must be the raw JSON object only, without any surrounding text, explanations, or markdown formatting like ```json.

**Output Format:**
{
  \"material\": \"[material]\",
  \"type\": \"[pole_type]\"

}

**Example:**

If the provided image shows a wooden electricity pole with a transformer, power lines, and a street lamp, your output should be:

{
  \"material\": \"wood\",
  \"type\": \"electricity pole\"

}
"""

## Main processing block

In [ ]:
results_list = []

try:
    # Fetch data from BigQuery
    BIGQUERY_SQL_QUERY = f"""
    SELECT
      gcs_uri
    FROM
      `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
      WHERE asset_type = "ASSET_CLASS_UTILITY_POLE"

    LIMIT {QUERY_LIMIT}
    """
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    eval_dataset = bigquery_client.query(BIGQUERY_SQL_QUERY).to_dataframe()
    print(f"Successfully fetched {len(eval_dataset)} records for evaluation.")

    # Loop through the dataset
    for index, row in eval_dataset.iterrows():
        uri = row['gcs_uri']
        print(f"--- ({index + 1}/{len(eval_dataset)}) Evaluating {uri} ---")

        flash_classification_raw = classify_image_with_gemini(gcs_uri=uri, prompt=prompt, model_name=MODEL_FLASH)
        pro_classification_raw = classify_image_with_gemini(gcs_uri=uri, prompt=prompt, model_name=MODEL_PRO)

        # Extract clean JSON strings
        flash_classification_str = _extract_json(flash_classification_raw)
        pro_classification_str = _extract_json(pro_classification_raw)

        # Parse into JSON objects with error handling
        try:
            flash_output_json = json.loads(flash_classification_str)
        except json.JSONDecodeError:
            flash_output_json = {'error': 'Failed to parse JSON from flash model.', 'raw_output': flash_classification_raw}

        try:
            pro_output_json = json.loads(pro_classification_str)
        except json.JSONDecodeError:
            pro_output_json = {'error': 'Failed to parse JSON from pro model.', 'raw_output': pro_classification_raw}

        judgement = judge_outputs(json.dumps(flash_output_json), json.dumps(pro_output_json))

        results_list.append({
            'gcs_uri': uri,
            'flash_output': flash_output_json,
            'pro_output': pro_output_json,
            'judgement': judgement
        })
        print(f"Verdict: {judgement}")

    results_df = pd.DataFrame(results_list)
    print("\n✅ Evaluation Complete. Analyzing results...")

    agreement_count = results_df['judgement'].str.startswith('SAME').sum()
    total_count = len(results_df)
    agreement_percentage = (agreement_count / total_count) * 100 if total_count > 0 else 0
    print(f"\n--- Aggregate Results ---")
    print(f"Overall Model Agreement: {agreement_percentage:.2f}%\n")

    disagreements_df = results_df[results_df['judgement'].str.startswith('DIFFERENT')].copy()
    print("--- Examples of Disagreements ---")

    if not disagreements_df.empty:
        pd.set_option('display.max_colwidth', None)
        display(disagreements_df.head(3))
    else:
        print("No disagreements found.")

except Exception as e:
    print(f"An error occurred during the evaluation pipeline: {e}")